<a href="https://colab.research.google.com/github/Kaiziferr/machine_learning/blob/main/decision_tree/02_pre_pruning_tree_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import warnings
from unicodedata import normalize

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import (
    train_test_split, GridSearchCV)

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import (mean_absolute_error, mean_squared_error)

# **Datos**
---

Conjunto de los volúmenes de gas suministrado por las Estaciones de Servicio en Colombia.
- The set of gas volumes supplied by gas stations in Colombia.



**Información de la Entidad**

- Área o dependencia: Dirección de Hidrocarburos
- Nombre de la Entidad: Ministerio de Minas y Energía
- Departamento: Bogotá D.C.
- Municipio: Bogotá D.C.
- Orden: Nacional
- Sector: Minas y Energía

**Información de Datos**

- Cobertura Geográfica: Nacional
- Frecuencia de Actualización: Diaria
- Fecha Emisión (aaaa-mm-dd): 2023-08-17

Suministró los datos: Ministerio de Minas y Energía

path data: https://www.datos.gov.co/Minas-y-Energ-a/Consulta-Ventas-de-Gas-Natural-Comprimido-Vehicula/v8jr-kywh/about_data

- FECHA_VENTA: fecha de la transacción (transaction date)
- ANIO_VENTA: fecha de la transacción (transaction year)
- MES_VENTA: fecha de la transacción (transaction month)
- DIA_VENTA: fecha de la transacción (transaction day)
- CODIGO_MUNICIPIO_DANE: Código del municipio (Municipality code (DANE))
- DEPARTAMENTO: departamento (department)
- MUNICIPIO: municipio (municipality)
- LATITUD: coordenadas de georeferenciación (georeferencing coordinates (latitude))
- LONGITUD: coordenadas de georeferenciación (georeferencing coordinates (longitude))
- TIPO_AGENTE: tipo del agente proveedor (type of supplier agent)
- TIPO_DE_COMBUSTIBLE: combustible suministrado (type of fuel supplied)
- EDS_ACTIVAS: estaciones de servicio activas (active service stations)
- NUMERO_DE_VENTAS: (number sales)
- VEHICULOS_ATENDIDOS: vehículos atendidos (number of vehicles serviced)
- CANTIDAD_VOLUMEN_SUMINISTRADO:  volume supplied during refueling

@By: **Steven Bernal**

@Nickname: **Kaiziferr**

@Git: https://github.com/Kaiziferr

# **Functions**
---

In [47]:
def def_group(
     function_dictionary:dict,
     filter_feature:list,
     new_val_col:list,
     data):
  try:
    data_group = data.groupby(
    filter_feature).aggregate(function_dictionary)

    data_group.columns =  new_val_col
    data_group = data_group.reset_index()
    return data_group
  except Exception as e:
    print(e)

# **Config**
---


In [48]:
sns.set(style="darkgrid")
pd.set_option('display.float_format', '{:,.2f}'.format)
title_data = 'Volumes of gas supplied by Service Stations in Colombia.'
random_seed = 12354
warnings.filterwarnings('ignore')

# **Data**
---

In [49]:
url_master =  'https://drive.google.com/file/d/1d2zxaI8riPA7SJm3cCw_jrrUYCIc_63F/view?usp=sharing'
url_materiales_mineros = 'https://drive.google.com/uc?id=' + url_master.split('/')[-2]
data_master = pd.read_csv(url_materiales_mineros, dtype='str')

The names features of data set are changed of spanish to english

In [50]:
set_columns = [
    'transaction_date',
    'year_transaction',
    'month_transaction',
    'day_transaction',
    'DANE_code',
    'department',
    'municipality',
    'latitude',
    'longitude',
    'type_supplier_agent',
    'type_fuel_supplied',
    'active_service_stations',
    'number_sales',
    'vehicles_served',
    'Volume_supplied_during_refuels']

data_master.columns = set_columns

In [51]:
data_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125771 entries, 0 to 125770
Data columns (total 15 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   transaction_date                125771 non-null  object
 1   year_transaction                125771 non-null  object
 2   month_transaction               125771 non-null  object
 3   day_transaction                 125771 non-null  object
 4   DANE_code                       125771 non-null  object
 5   department                      125771 non-null  object
 6   municipality                    125771 non-null  object
 7   latitude                        125771 non-null  object
 8   longitude                       125771 non-null  object
 9   type_supplier_agent             125771 non-null  object
 10  type_fuel_supplied              125771 non-null  object
 11  active_service_stations         125771 non-null  object
 12  number_sales                  

In [52]:
data_master.columns

Index(['transaction_date', 'year_transaction', 'month_transaction',
       'day_transaction', 'DANE_code', 'department', 'municipality',
       'latitude', 'longitude', 'type_supplier_agent', 'type_fuel_supplied',
       'active_service_stations', 'number_sales', 'vehicles_served',
       'Volume_supplied_during_refuels'],
      dtype='object')

In [53]:
data_master.head()

,transaction_date,year_transaction,month_transaction,day_transaction,DANE_code,department,municipality,latitude,longitude,type_supplier_agent,type_fuel_supplied,active_service_stations,number_sales,vehicles_served,Volume_supplied_during_refuels
0,2022-06-17,2022,06,17,68682,SANTANDER,FLORIDABLANCA,7.0797047615,-73.0679931641,ESTACION DE SERVICIO DE GNCV,GNV,2,671,576,4909.3
1,2020-07-13,2020,07,13,23230,CORDOBA,MONTERIA,8.5846977234,-75.950553894,ESTACION DE SERVICIO DE GNCV,GNV,1,261,168,1413.75
2,2022-04-16,2022,04,16,85850,CASANARE,YOPAL,5.2427449226,-72.258026123,ESTACION DE SERVICIO DE GNCV,GNV,7,1162,560,8883.95
3,2022-06-06,2022,06,06,68680,SANTANDER,BUCARAMANGA,7.1558337212,-73.1115722656,ESTACION DE SERVICIO DE GNCV,GNV,9,1957,1331,13073.23
4,2023-02-06,2023,02,06,13138,BOLIVAR,TURBACO,10.3531074524,-75.3797149658,ESTACION DE SERVICIO DE GNCV,GNV,1,63,53,660.86


The real data types are assigned to the data.

In [54]:
data_master[[
    'year_transaction',
    'month_transaction',
    'day_transaction',
    'active_service_stations',
    'number_sales',
    'vehicles_served',
]] = data_master[[
    'year_transaction',
    'month_transaction',
    'day_transaction',
    'active_service_stations',
    'number_sales',
    'vehicles_served',
]].astype('int32')

data_master[[
    'Volume_supplied_during_refuels',
]] = data_master[[
    'Volume_supplied_during_refuels',
]].astype('float64')

One year is select, this year is 2023

In [55]:
data_train = data_master[
    (data_master['year_transaction']>=2022) & (data_master['year_transaction']<2023)]
data_test = data_master[data_master['year_transaction']==2024]

A Key is created, to identify the register. This key is the merge of the municipality and department

In [56]:
data_train['key'] = data_train.department.str.cat(
    data_train.municipality, sep='-')

In [57]:
data_train.columns

Index(['transaction_date', 'year_transaction', 'month_transaction',
       'day_transaction', 'DANE_code', 'department', 'municipality',
       'latitude', 'longitude', 'type_supplier_agent', 'type_fuel_supplied',
       'active_service_stations', 'number_sales', 'vehicles_served',
       'Volume_supplied_during_refuels', 'key'],
      dtype='object')

Grouped by the jurisdiction key and amounts  to be able to sum the next feautres:


*   Volume_supplied_during_refuels
*   vehicles_served
*   number_sales
*   active_service_stations



In [58]:
function_dictionary = {
    "Volume_supplied_during_refuels": ['sum'],
    "vehicles_served": ['sum'],
    'number_sales': ['sum'],
    'active_service_stations': ['sum']
}

filter_feature = [
    'key',
    'month_transaction']


new_val_col  = [
    "Volume_supplied_during_refuels",
    "vehicles_served",
    'number_sales',
    'active_service_stations'
]

data_group = def_group(
    function_dictionary,
    filter_feature,
    new_val_col,
    data_train
)
data_group.head()

,key,month_transaction,Volume_supplied_during_refuels,vehicles_served,number_sales,active_service_stations
0,ANTIOQUIA-APARTADO,1,"37,935.13",1364,1855,30
1,ANTIOQUIA-APARTADO,2,"33,083.70",1214,1682,25
2,ANTIOQUIA-APARTADO,3,"38,113.29",1497,2079,31
3,ANTIOQUIA-APARTADO,4,"38,372.85",1445,2092,30
4,ANTIOQUIA-APARTADO,5,"35,000.06",1318,1881,28


The theoretical average of the refueled volume, vehicles attended, and number of sales is calculated. The theoretical average represents the variables' activity relative to uptime.

In [59]:
function_dictionary = {
    "Volume_supplied_during_refuels": ['mean'],
    "vehicles_served": ['mean'],
    'number_sales': ['mean'],
    'active_service_stations': ['sum'],
    'month_transaction': ['count']
}

filter_feature = [
    'key'
]


new_val_col  = [
    "Volume_supplied_during_refuels_mean",
    "vehicles_served_mean",
    'number_sales_mean',
    'active_service_stations_sum',
    'active months'
]

data_group2 = def_group(
    function_dictionary,
    filter_feature,
    new_val_col,
    data_group
)

In [60]:
data_group2

,key,Volume_supplied_during_refuels_mean,vehicles_served_mean,number_sales_mean,active_service_stations_sum,active months
0,ANTIOQUIA-APARTADO,"31,949.73","1,257.83","1,739.50",353,12
1,ANTIOQUIA-BARBOSA,"284,475.96","1,652.25","3,551.67",363,12
2,ANTIOQUIA-BELLO,"163,416.86","12,219.75","14,173.92",1149,12
3,ANTIOQUIA-CAUCASIA,"39,506.04","2,322.00","3,564.08",362,12
4,ANTIOQUIA-ENVIGADO,"65,459.14","6,523.17","6,874.83",365,12
...,...,...,...,...,...,...
97,VALLE DEL CAUCA-JAMUNDI,"13,235.31","1,308.75","1,513.62",170,8
98,VALLE DEL CAUCA-PALMIRA,"63,792.69","6,543.70","8,287.00",807,10
99,VALLE DEL CAUCA-ROLDANILLO,"11,544.53",859.00,"1,004.00",23,1
100,VALLE DEL CAUCA-TULUA,"153,590.34","14,681.75","21,324.50",1044,12


# **Data Split**
---

The data is divided into predictors and labels

In [61]:
X = data_group2[["vehicles_served_mean", "number_sales_mean", "active_service_stations_sum", "active months"]]
y = data_group2['Volume_supplied_during_refuels_mean']

The data is divided into training data and test data

In [62]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    train_size=0.75,
    random_state=random_seed
)

**Pre Pruning, First Way**

First, a model will be instantiated, where only the seed will be adjusted. Then, a grid will be implemented with the following parameters:

- criterion: The function to measure the quality of a split
- max_depth: The maximum depth of the tree
- max_features: The number of features to consider when looking for the best split
- min_samples_split: The minimum number of samples required to split an internal node

The GridSearchCV method will be implemented using the scoring neg_mean_absolute_error, with a fold ten cross-validation splitting strategy. The model will be retrained with the best score and returning the training scores

In [63]:
dict_params = {
    'criterion': ['squared_error', 'mse_friedman', 'absolute_error', 'poisson'],
    'max_depth': [3,6,9,12],
    'max_features': [None, 'sqrt', 'log2', 0.75],
    'min_samples_split': [2,4,8,10]
}


model_regresor_prepruning_v1 = DecisionTreeRegressor(random_state=random_seed)
model_regresor_prepruning_v1 = GridSearchCV(
    estimator = model_regresor_prepruning_v1,
    param_grid = dict_params,
    scoring = 'neg_mean_absolute_error',
    cv = 10,
    refit = True,
    return_train_score = True
)

model_regresor_prepruning_v1.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(random_state=12354),
             param_grid={'criterion': ['squared_error', 'mse_friedman',
                                       'absolute_error', 'poisson'],
                         'max_depth': [3, 6, 9, 12],
                         'max_features': [None, 'sqrt', 'log2', 0.75],
                         'min_samples_split': [2, 4, 8, 10]},
             return_train_score=True, scoring='neg_mean_absolute_error')

In [64]:
model_regresor_prepruning_v1_best = model_regresor_prepruning_v1.best_estimator_
print(f"Tree depth: {model_regresor_prepruning_v1_best.get_depth()}")
print(f"Number of terminal nodes: {model_regresor_prepruning_v1_best.get_n_leaves()}")


Tree depth: 9
Number of terminal nodes: 36


In [65]:
scores = pd.DataFrame(model_regresor_prepruning_v1.cv_results_)
scores.sort_values(by="mean_test_score", ascending=False)[['mean_train_score', 'mean_test_score', 'params']]

,mean_train_score,mean_test_score,params
225,"-20,441.09","-67,390.22","{'criterion': 'poisson', 'max_depth': 9, 'max_..."
168,"-1,035.12","-67,592.59","{'criterion': 'absolute_error', 'max_depth': 9..."
164,"-1,035.12","-67,592.59","{'criterion': 'absolute_error', 'max_depth': 9..."
237,"-19,932.90","-68,189.11","{'criterion': 'poisson', 'max_depth': 9, 'max_..."
241,"-20,346.16","-68,890.62","{'criterion': 'poisson', 'max_depth': 12, 'max..."
...,...,...,...
123,NaN,NaN,"{'criterion': 'mse_friedman', 'max_depth': 12,..."
124,NaN,NaN,"{'criterion': 'mse_friedman', 'max_depth': 12,..."
125,NaN,NaN,"{'criterion': 'mse_friedman', 'max_depth': 12,..."
126,NaN,NaN,"{'criterion': 'mse_friedman', 'max_depth': 12,..."


In [66]:
print(f'The best score: {-model_regresor_prepruning_v1.best_score_:,.2f}')

The best score: 67,390.22


In [67]:
print(f'The best params: {model_regresor_prepruning_v1.best_params_}')

The best params: {'criterion': 'poisson', 'max_depth': 9, 'max_features': None, 'min_samples_split': 4}


In [68]:
y_predict_prepruning_v1 = model_regresor_prepruning_v1_best.predict(X_test)

In [69]:
print(f'Mean Absolute Error {mean_absolute_error(y_test, y_predict_prepruning_v1):,.2f}')
print(f'Mean Squared  Error {mean_squared_error(y_test, y_predict_prepruning_v1):,.2f}')
print(f'Root Mean Squared Error {mean_squared_error(y_test, y_predict_prepruning_v1, squared=False):,.2f}')

Mean Absolute Error 519,203.11
Mean Squared  Error 6,299,369,437,979.88
Root Mean Squared Error 2,509,854.47


In [70]:
y.describe()

,Volume_supplied_during_refuels_mean
count,102.00
mean,"281,561.40"
std,"1,404,634.13"
min,10.62
25%,"18,327.37"
50%,"54,926.33"
75%,"125,413.10"
max,"14,063,336.04"


The error is below the average of the data.

**Pre Pruning, Second Way**

The second option to perform prepruning is through of "alphas" estimation. 80 alphas were generated on a scale from 0 to 200,000. Usualy, for regression problems, it's important to use values within the same magnitude of data value. Defining the minimum or maximum value depends on the iterative process to determine the best performance.

In [71]:
dict_params = {
    'ccp_alpha': np.linspace(0, 200000, 80)
}

Implemented a grid with the metric 'neg_mean_absolute_error', 10-fold cross-validation, refitting the best model, and return the training scores

In [72]:
model_regresor_prepruning_v2 = DecisionTreeRegressor(random_state=random_seed)
model_regresor_prepruning_v2 = GridSearchCV(
    estimator = model_regresor_prepruning_v2,
    param_grid = dict_params,
    scoring = 'neg_mean_absolute_error',
    cv = 10,
    refit = True,
    return_train_score = True
)

model_regresor_prepruning_v2.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(random_state=12354),
             param_grid={'ccp_alpha': array([     0.        ,   2531.64556962,   5063.29113924,   7594.93670886,
        10126.58227848,  12658.2278481 ,  15189.87341772,  17721.51898734,
        20253.16455696,  22784.81012658,  25316.4556962 ,  27848.10126582,
        30379.74683544,  32911.39240506,  35443.03797468,  37974.6835443 ,
        40506.32911392,  43037.97468...
       151898.73417722, 154430.37974684, 156962.02531646, 159493.67088608,
       162025.3164557 , 164556.96202532, 167088.60759494, 169620.25316456,
       172151.89873418, 174683.5443038 , 177215.18987342, 179746.83544304,
       182278.48101266, 184810.12658228, 187341.7721519 , 189873.41772152,
       192405.06329114, 194936.70886076, 197468.35443038, 200000.        ])},
             return_train_score=True, scoring='neg_mean_absolute_error')

In [73]:
model_regresor_prepruning_v2_best = model_regresor_prepruning_v2.best_estimator_
print(f"Tree depth: {model_regresor_prepruning_v2_best.get_depth()}")
print(f"Number of terminal nodes: {model_regresor_prepruning_v2_best.get_n_leaves()}")

Tree depth: 11
Number of terminal nodes: 61


In [74]:
scores = pd.DataFrame(model_regresor_prepruning_v2.cv_results_)
scores.sort_values(by="mean_test_score", ascending=False)[['mean_train_score', 'mean_test_score', 'params']]

,mean_train_score,mean_test_score,params
14,-156.33,"-88,736.65",{'ccp_alpha': 35443.037974683546}
13,-153.06,"-88,744.35",{'ccp_alpha': 32911.39240506329}
12,-153.06,"-88,744.35",{'ccp_alpha': 30379.74683544304}
8,-103.11,"-88,746.51",{'ccp_alpha': 20253.164556962027}
10,-127.78,"-88,746.51",{'ccp_alpha': 25316.455696202534}
...,...,...,...
75,-525.47,"-88,938.49",{'ccp_alpha': 189873.417721519}
76,-525.47,"-88,938.49",{'ccp_alpha': 192405.06329113926}
77,-525.47,"-88,938.49",{'ccp_alpha': 194936.7088607595}
78,-525.47,"-88,938.49",{'ccp_alpha': 197468.35443037975}


In [75]:
print(f'The best score: {-model_regresor_prepruning_v2.best_score_:,.2f}')

The best score: 88,736.65


In [76]:
print(f'The best params: {model_regresor_prepruning_v2.best_params_}')

The best params: {'ccp_alpha': 35443.037974683546}


In [77]:
y_predict_prepruning_v2 = model_regresor_prepruning_v2_best.predict(X_test)

In [78]:
print(f'Mean Absolute Error {mean_absolute_error(y_test, y_predict_prepruning_v2):,.2f}')
print(f'Mean Squared  Error {mean_squared_error(y_test, y_predict_prepruning_v2):,.2f}')
print(f'Root Mean Squared Error {mean_squared_error(y_test, y_predict_prepruning_v2, squared=False):,.2f}')

Mean Absolute Error 545,034.22
Mean Squared  Error 6,763,305,018,048.65
Root Mean Squared Error 2,600,635.50


The error is below the average of the data. The second method has an error below the average when using "MAE"; but it is above the error of the first method. The second option tends to be a better alternative than the first; however, it is more complex to identify the range of values for the "alphas.

@By: **Steven Bernal**

@Nickname: **Kaiziferr**

@Git: https://github.com/Kaiziferr